<a href="https://colab.research.google.com/github/dadler6/info5610_studentlife/blob/master/INFO_5610_Sept_24_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

We are going to imitate a feature in the [HealthRhythms API](https://rest.docs.healthrhythms.com/#/operations/get_summary_v2_data_summary_get) using location data from the public [StudentLife](https://studentlife.cs.dartmouth.edu/dataset.html) dataset. Remember, we read [the StudentLife paper in class](https://dl.acm.org/doi/10.1145/2632048.2632054). We will see if the feature we create is associated with a stress EMA  response.

Let's start by importing some helpful data manipulation libraries.

In [ ]:
# For data manipulation
import pandas as pd # Pandas
import numpy as np # Numpy

# Uploading data

We can upload raw StudentLife data using the code below. I put the data we will use in a public GitHub repository, [linked](https://github.com/dadler6/info5610_studentlife).

In [ ]:
# URL to github
github_url = "https://raw.githubusercontent.com/dadler6/info5610_studentlife/master/"

# Participant IDs
participant_ids = [
    'u00', 'u10', 'u20', 'u33', 'u45', 'u56',
    'u01', 'u12', 'u22', 'u34', 'u46', 'u57',
    'u02', 'u13', 'u23', 'u35', 'u47', 'u58',
    'u03', 'u14', 'u24', 'u36', 'u49', 'u59',
    'u04', 'u15', 'u25', 'u39', 'u50',
    'u05', 'u16', 'u27', 'u41', 'u51',
    'u07', 'u17', 'u30', 'u42', 'u52',
    'u08', 'u18', 'u31', 'u43', 'u53',
    'u09', 'u19', 'u32', 'u44', 'u54']

# Upoload data
data = dict()
for data_type in ['gps', 'Stress']:
    # Create dict
    data[data_type] = []
    # Check data_type and append file type
    for i in participant_ids:
        # GPS is csv
        if data_type == 'gps':
            df = pd.read_csv(
                github_url + data_type + '/' + \
                data_type + '_' + i + '.csv'
            )
        # Stress is JSON
        else:
            df = pd.read_json(
                github_url + data_type + '/' + \
                data_type + '_' + i + '.json'
            )
        # Add Participant ID
        df['pid'] = i
        # Save
        data[data_type].append(df)

Now let's concatenate the data, see how large of a dataset we have, and print the first 5 rows of data.

In your own project, it may also be good to look at the number of unique participants, as well as how much data exists per participant.

**What else may be good to understand when you initially upload data?**

In [ ]:
# GPS
data['gps'] = pd.concat(data['gps']).reset_index()
print('Shape', data['gps'].shape)
data['gps'].head()

In [ ]:
# Stress
data['Stress'] = pd.concat(data['Stress']).reset_index()
print('Shape', data['Stress'].shape)
data['Stress'].head()

# Data Cleaning

It looks like some of the columns are not aligned wtih the data contained within the column! Let's clean them up a bit.

In [ ]:
# In the GPS data, it appears the time was an index, initially,
# which shifted when I ran the reset_index() function.
# Lets retrieve the columns we are interested in and name them something more ideal...

data['gps'] = data['gps'][['pid', 'index', 'accuracy', 'latitude']]
data['gps'].columns = ['pid', 'datetime', 'latitude', 'longitude']

In [ ]:
# In the Stress data, as per the documentation, we only want to keep the
# "resp_time", and "level" columns
data['Stress'] = data['Stress'][['pid', 'resp_time', 'level', 'location']]

# Let's also rename the columns such that the names align with
# GPS data, and we can give the "level" column a more descriptive name
data['Stress'].columns = ['pid', 'datetime', 'ema_stress', 'location']

We also need to extract relevant datetime information. The documentation says that the datetime is a UNIX time. We can localize the time by using the location data in each dataframe.

We'll use the `timezonefinder` library to do this. Let's install it in our kernel and then import it.


In [ ]:
# For datetime
!pip install timezonefinder
import timezonefinder

We can create a timezonefinder object.

In [ ]:
tz_finder = timezonefinder.TimezoneFinder()

And get the timzeones of the GPS data to localize it.

In [ ]:
data['gps']['timezone'] = [tz_finder.timezone_at(
    lng=data['gps'].loc[ind, 'longitude'],
    lat=data['gps'].loc[ind, 'latitude']
) for ind in data['gps'].index]

The Stress datetime column is a little trickier to manipulate because the location is listed as a string in a single column, separated by a comma. We'll have to separate the string first and extract the latitude/longitude coordinates.

In addition, there are some NA values in this column. Let's drop those for the purposes of this particular lecture, but you may want to think about missing data filling for your project.

**What are the benefits/downsides of filling missing Stress EMA data? There may be GPS data that aligns with the missing data...**

In [ ]:
# Drop NA rows
data['Stress'] = data['Stress'].dropna()

# There are also some rows where location is not known
# filter these out
data['Stress'] = data['Stress'].loc[
    data['Stress']['location'] != 'Unknown', :
]

# Reset the index and create a new column
data['Stress'] = data['Stress'].reset_index(drop=True)
data['Stress']['timezone'] = None


# Iterate through each remaining index
for ind in data['Stress'].index:
    # Split value by comma
    lat, lon = data['Stress'].loc[ind, 'location'].split(',')
    # Get timezone
    data['Stress'].loc[ind, 'timezone'] = \
        tz_finder.timezone_at(
            lng=float(lon), lat=float(lat)
        )

One other funky thing about the stress data. This is the current meaning of the values in this column:

1. A little stressed
2. Definitely stressed
3. Stressed out
4. Feeling good
5. Feeling great

We should map these to values that are a bit more...ordered. We can perform the following mapping to simplify this problem a bit, between feeling good/great versus definitely stressed/stress out.

The "a little stressed" response is a bit ambiguous. Only analyzing extremes will make our lives easier for today.

In [ ]:
data['Stress']['ema_stress'] = data['Stress']['ema_stress'].map({
    5: 0,
    4: 0,
    1: None,
    2: 1,
    3: 1
})

data['Stress'] = data['Stress'].dropna().reset_index(drop=True)

## Datetime manipulation and localization

Now that the columns are cleaned, let's localize the time. First, we'll check the dtypes of each dataframe, to see if Pandas has inferred a datetime object previously.

In [ ]:
data['gps'].dtypes, data['Stress'].dtypes

It looks like Pandas has inferred a datetime object for the Stress EMA data, but not for the GPS data. We'll have to first convert the gps data to a datetime before localizing the timezone.

In [ ]:
# Iterate through each df
for data_type in data.keys():
    # If GPS, create datetime
    if data_type == 'gps':
        # In UNIX seconds timestamp
        data['gps']['datetime'] = pd.to_datetime(data['gps']['datetime'], unit='s')
    # Now localize
    data[data_type]['loc_datetime'] = None
    for ind in data[data_type].index:
        data[data_type].loc[ind, 'loc_datetime'] = \
            data[data_type].loc[ind, 'datetime'].tz_localize("UTC").tz_convert(
                data[data_type].loc[ind, 'timezone'])

# Feature Engineering

Now that we have two dataframes, let's create a feature! We will focus on the feature `activity_percent`, which measures the percentage of time active during the day.

We'll use the feature creation methods specified in [this paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4526997/).

## Activity Percentage

We'll calculate "activity_percent" similar to the "Transition Time" listed in the paper. The paper states.

"Transition time represented the percentage of time during which a participant was in a non-stationary state (see Data Preprocessing). This was calculated by dividing the number of GPS location samples in transition states by the total number of samples."

To find non-stationary states, the paper does the following:

"To do so, we estimated the movement speed at each location data sample by calculating its time derivative and then used a threshold speed that defined the boundary between these two states. In this study, we set this threshold to 1 km/h."

Let's define a function together called activity_percent to identify the percentage of "active" data points (moving >1 km/h) using the latitude, longitude columns. The function will output the percentage of active data points.

To do this, we'll also need to calculate the distance between two latitude, longitude points, which we can approximate using the [Haversine Formula](https://en.wikipedia.org/wiki/Haversine_formula).

**How could this feature be improved?**

**When you build a feature, how can you check that the function performs correctly?**

In [ ]:
def distance(origin, destination):
    """
    Compute distance between two points

    :param origin: (<float>, <float>) lat, long location origin
    :param destination: (<float>, <float>) lat, long location destination

    :return: <float> the distance in km between points
    """
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = np.radians(lat2-lat1)
    dlon = np.radians(lon2-lon1)
    a = np.sin(dlat/2) * np.sin(dlat/2) + np.cos(np.radians(lat1)) \
        * np.cos(np.radians(lat2)) * np.sin(dlon/2) * np.sin(dlon/2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = radius * c

    # Return distances in kilometers
    return d

def activity_percent(df):
    """
    Calculates the percentage of data points "in transition".
    "In transition" defined as moving >1 km/hour

    :param df: pd.DataFrame, GPS data for a specific user
                             with latitude, longitude columns,
                             and loc_datetime column represented
                             the localized datetime
    :return: float, the percentage of data points in transition
    """
    # Sort dataframe
    df = df.sort_values(by='loc_datetime').reset_index(drop=True)

    # Make distance, timediff, moving cols
    df['dist'] = None
    df['timediff'] = None

    # Iterate through rows after
    # first index
    for ind in df.index[1:]:
        # Get information
        loc1 = df.loc[ind - 1, ['latitude', 'longitude']]
        loc2 = df.loc[ind, ['latitude', 'longitude']]
        t1 = df.loc[ind - 1, 'loc_datetime']
        t2 = df.loc[ind, 'loc_datetime']

        # Get distance
        df.loc[ind, 'dist'] = distance(loc1, loc2)
        # Get timediff in hours
        df.loc[ind, 'timediff'] = (t2 - t1).seconds / 3600

    # Drop na
    df = df.dropna().reset_index(drop=True)

    # Get speed
    df['speed'] = df['dist'] / df['timediff']

    # > 1 km/hour = moving
    df['moving'] = 0
    df.loc[df['speed'] > 1, 'moving'] = 1

    # Return active_ime
    return df['moving'].mean()

To create some features, we need to make sure they align with the outcome data, in this case the stress data. We'll also need to choose a retrospective time window in which to create features within.

The Stress EMA states "Right now, I am...", so it is measuring momentary stress. Therefore, we should probably use location data pretty close to the timeframe to assess the answer to this question.

For convenience, maybe we'll use 8 hours of data prior to each stress EMA answered to respond to this question. We'll need the datetime library to manipulate the dates. Let's import it.

**What may be better ways to define a retrospective time window??**

In [ ]:
from datetime import timedelta

Now let's iterate through each stress EMA recorded, and estimate the active time in the 8 hours before each stress EMA was recorded.

In [ ]:
# For holding cleaned data
cleaned_dfs = []

# Iterate through each pid
for pid in data['Stress']['pid'].unique():
    # Get data for a pid
    stress_df = data['Stress'].loc[data['Stress']['pid'] == pid, :]
    gps_df = data['gps'].loc[data['gps']['pid'] == pid, :]

    # Reset index
    stress_df = stress_df.reset_index(drop=True)
    gps_df = gps_df.reset_index(drop=True)

    # Create active time col
    stress_df['activity_percent'] = None

    # Go through each stress EMA answered
    for ind in stress_df.index:
        # Get datetime of the stress response
        end_dt = stress_df.loc[ind, 'loc_datetime']
        # Get 8 hours before this datetime
        start_dt = end_dt - timedelta(hours=8)

        # Get location data with this datetime
        gps_df_temp = gps_df.loc[
            (gps_df.loc_datetime >= start_dt) &
            (gps_df.loc_datetime <= end_dt)
        ]
        # Check to see if data exists, if it does create features
        if gps_df_temp.shape[0] > 0:
            stress_df.loc[ind, 'activity_percent'] = activity_percent(df=gps_df_temp.copy())

    # Save stress_df
    cleaned_dfs.append(
        stress_df[['pid', 'loc_datetime', 'ema_stress', 'activity_percent']].dropna()
    )

Now let's concatenate together the cleaned DataFrames, and see how much data we have left.

In [ ]:
# Concatenate
cleaned_df = pd.concat(cleaned_dfs).reset_index(drop=True)
# Make sure column is a flot
cleaned_df['activity_percent'] = cleaned_df['activity_percent'].astype(float)
# Check results
print('Shape', cleaned_df.shape)
cleaned_df.head()

# Analysis

There are many different ways to analyze data, including visualizations, statistical models, and machine learning models.

## Visuals

Let's create some simple visuals to analyze our data. First, let's simply plot the average activity percentage per binary stress category.

We can import the pyplot as seaborn libraries as our visualization libraries.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

Now, let's visualize the distribution of `activity_percent` per stress response value. We'll also clean up the labels for readability.

In [ ]:
def plot_activity_percent(df):
    """
    Plot activity percent boxplots

    :param df: pd.DataFrame, the df to plot
               with ema_stress and activity_percent
               columns
    """
    # Create figure
    plt.figure(figsize=(5, 3))

    # Plot boxplot
    sns.boxplot(
        x=df['ema_stress'],
        y=df['activity_percent'],
    )

    # Label axes
    plt.xlabel('Stress EMA')
    plt.ylabel('Active Time')

    sns.despine()

In [ ]:
plot_activity_percent(cleaned_df)

Hmm, it's kind of hard to see differences. Maybe we can look at data for specific individuals who have a lot of data.

In [ ]:
plot_activity_percent(cleaned_df.loc[cleaned_df.pid == 'u59', :])

In [ ]:
plot_activity_percent(cleaned_df.loc[cleaned_df.pid == 'u16', :])

## Statistics

It looks like some differences exist at an individual level. Maybe we can quantify these differences using a regression library.

We'll use the [statsmodels logit](https://www.statsmodels.org/stable/generated/statsmodels.discrete.discrete_model.Logit.html) function, which calculates logistic regression coefficients.

In [ ]:
# Import
from statsmodels.discrete.discrete_model import Logit

# Calculate coefficient
cleaned_df['bias'] = 1
logit = Logit(
    endog=cleaned_df['ema_stress'],
    exog=cleaned_df[['activity_percent', 'bias']]
)
# Run
res = logit.fit()

# Print summary
res.summary()

It looks like there is a significant difference! Logistic regression coefficeints are a bit difficult to interpret.  

We can transform these differences into [odds ratios](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2938757/), which are a bit more interpretable. Odds ratios, in this case, would represent the expected activity percentage for stressed individuals divided by the expected activity percentage for non-stressed individuals.

In [ ]:
oddsratio = np.exp(res.params).loc['activity_percent']
conf_int = np.exp(res.conf_int()).loc['activity_percent', :]

pd.Series(
    [oddsratio, conf_int.loc[0], conf_int.loc[1]],
    index=['OR', '2.75%', '97.5%'],
)

In this example, the estimated activity percentage of someone experiencing stress is OR=0.08 95% CI (0.025 -- 0.28) times the activity percentage of someone not experiencing stress.

**Based upon the visuals, is this the coefficient you would expect?**

### Accounting for repeated measures

One thing to account for in this setting is that we technically do not have independently, identically distributed variables (called IID), and thus logistic regression or similar linear models should not be used unless we somehow account for correlated measures.

Generalized estimating equations (GEE) is a similar type of linear model that allows us to model correlations between subjects, and thus better estimate the population-wide coefficients. We can use the statsmodels [GEE implementation](https://www.statsmodels.org/stable/gee.html).

In [ ]:
from statsmodels.genmod.generalized_estimating_equations import GEE
import statsmodels.api as sm

# We will need to tell that model that "pid" is a grouping variable
# and that the response variable is binary (Binomial)

# Calculate coefficient
gee = GEE(
    endog=cleaned_df['ema_stress'],
    exog=cleaned_df[['activity_percent', 'bias']],
    groups=cleaned_df['pid'],
    family=sm.families.Binomial(),
    cov_struct=sm.cov_struct.Exchangeable() # The covariance model
)

# For more on covariance models, see:
# https://online.stat.psu.edu/stat504/lesson/12/12.1

# Run
gee_res = gee.fit()

# Print summary
gee_res.summary()

Notice how the GEE coefficeints are not significant!

**Why is this??** Think: we want the regression coefficient to approximate the population effect independent of any specific individual in the population...

Thinking about the below data may help...

| pid | activity_percent | ema_stress |
|-|-|-|
|0|0.50|0|
|0|0.50|0|
|0|0.45|1|
|1|0.10|1|
|1|0.10|1|
|1|0.15|0|

# Your turn

Use this time to come up with other ways to visualize or model the activity percentage that may provide a better fit. A few thoughts:

* Based upon our visualizations, it looks like activity percentage changes with stress, but maybe not in the same direction per individual. How would you model this? What other analyses would you perform to focus more on individual-level differences?
* Feel free to improve the feature, as well, or create other features from the [HealthRhythms API](https://rest.docs.healthrhythms.com/#/operations/get_summary_v2_data_summary_get).

In [ ]:
# More code here